# LiDAR Data Processing with the Dataloop SDK

This notebook provides a comprehensive guide on preparing and managing LiDAR data on the Dataloop platform using its Python SDK. The workflow covers preparing a dataset with 3D Point Cloud Data (PCD) and 2D camera images, constructing a composite LiDAR video item, and uploading 3D annotations.

### Table of Contents
1. [Install Dependencies](#install-dependencies)
2. [Import Required Libraries](#import-libraries)
3. [Set Up Dataloop Environment](#setup-environment)
4. [LiDAR Data Preparation](#data-preparation)
    * [4.1 Local File Structure](#local-file-structure)
    * [4.2 Upload Data to Dataloop](#upload-data)
5. [Construct LiDAR Video Item](#construct-lidar-video)
6. [Upload Annotations](#upload-annotations)
    * [6.1 Define and Upload a 3D Cuboid](#upload-cuboid)
    * [6.2 View in Dataloop Studio](#view-in-studio)
7. [Conclusion and Next Steps](#conclusion)

For more detailed information, refer to the official Dataloop documentation:
- [LiDAR Data Setup](https://docs.dataloop.ai/docs/lidar-data-setup)
- [LiDAR Video Annotations](https://developers.dataloop.ai/tutorials/annotations/lidar/chapter)

Let's get started!

## <a id='install-dependencies'></a>1. Install Dependencies

First, let's ensure all required Python packages are installed. The cell below will install `dtlpy` for Dataloop SDK interaction and `dtlpylidar` for LiDAR-specific functionalities. It is recommended to run this to ensure you have the latest compatible versions.

In [ ]:
!pip install dtlpy git+https://github.com/dataloop-ai-apps/dtlpy-lidar.git --upgrade --quiet

## <a id='import-libraries'></a>2. Import Required Libraries

With the dependencies installed, we will now import the necessary libraries for this tutorial.

In [ ]:
import dtlpy as dl
from dtlpylidar.parsers.base_parser import LidarFileMappingParser

## <a id='setup-environment'></a>3. Set Up Dataloop Environment

To begin, we need to connect to the Dataloop platform. If you're not already logged in, running the cell below will prompt you to do so. We will then create a new project or retrieve an existing one to work in.

In [ ]:
if dl.token_expired():
    dl.login()

PROJECT_NAME = "your-lidar-project-name"
project = dl.projects.create(project_name=PROJECT_NAME)

> **Action Required:** In the cell above, replace `"your-lidar-project-name"` with your Dataloop project name. If a project with this name already exists, it will be retrieved; otherwise, a new one will be created.

## <a id='data-preparation'></a>4. LiDAR Data Preparation

To construct a LiDAR video item, your dataset must contain three key components: PCD files for 3D scenes, camera images for 2D views, and a `mapping.json` file for sensor calibration. This section explains how to structure these files locally before uploading them to Dataloop.

### <a id='local-file-structure'></a>4.1. Local File Structure

Before uploading to the Dataloop platform, your LiDAR data must be arranged in a specific local directory structure. This ensures that all components (point clouds, images) are correctly associated.

1.  **`lidar` folder:** Contains all PCD files, named numerically in sequence (e.g., `0.pcd`, `1.pcd`, ...).

2.  **`frames` folder:** Contains subfolders for each frame's camera images.
    *   Subfolders must be named numerically to match the PCD files (e.g., `0`, `1`, ...).
    *   Image files within each subfolder must be named sequentially (e.g., `0.jpg`, `1.jpg`, ...).

3.  **`mapping.json` file:** This file, located at the root of your data directory, contains the calibration data that links the 3D point clouds to the 2D images.

The `mapping.json` file must follow a specific schema. Below is a template demonstrating the required structure, including paths, timestamps, and camera calibration parameters (intrinsics and extrinsics).

```json
{
    "frames": {
        "0": {
            "path": <>, // for frame 0: "lidar/0.pcd" (Relative path from the mapping.json file)
            "timestamp": <>,
            "position": { // LiDAR sensor location (used as the center of the world)
                "x": 0.0,
                "y": 0.0,
                "z": 0.0
            },
            "heading": { // LiDAR sensor rotation (Quaternion)
                "x": 0.0,
                "y": 0.0,
                "z": 0.0,
                "w": 1.0
            },
            "images": { // if no images are provided, add an empty dict
                "0": {
                    "image_path": <>, // for frame 0 image 0: "frames/0/0.jpg" (Relative path from the mapping.json file)
                    "timestamp": <>,
                    "intrinsics": { // camera intrinsic
                        "fx": <>, // Focal length in pixels.
                        "fy": <>,
                        "cx": <>, // Optical center (the principal point), in pixels.
                        "cy": <>,
                    },
                    "extrinsics": { // camera extrinsic
                        "translation": { // camera location in world coordinates (in relation to the lidar sensor)
                            "x": <>,
                            "y": <>,
                            "z": <>
                        },
                        "rotation": { // rotation of the camera (Quaternion)
                            "w": <>,
                            "x": <>,
                            "y": <>,
                            "z": <>
                        }
                    },
                    "distortion" : { // distortion parameters
                        "k1": <>,
                        "k2": <>,
                        "p1": <>,
                        "p2": <>,
                        "k3": <>,
                        "k4": <>
                    }
                }
            }
        }
    }
}
```

### <a id='upload-data'></a>4.2. Upload Data to Dataloop

You can use your own data structured as described above, or use sample data from [OSDaR23](https://data.fid-move.de/dataset/osdar23) for this tutorial.

> **Action Required:** In the cell below, update `DATASET_NAME` with your desired dataset name and `DATA_PATH` with the local path to your data directory.

In [ ]:
DATASET_NAME = "My-LiDAR-Dataset"
DATA_PATH = "./data/*"
dataset = project.datasets.create(dataset_name=DATASET_NAME)
dataset.items.upload(local_path=DATA_PATH)

## <a id='construct-lidar-video'></a>5. Construct LiDAR Video Item

After uploading your raw data files (`.pcd`, `.jpg`, `mapping.json`), we use the Dataloop LiDAR SDK to create a single, composite LiDAR video item. The `LidarFileMappingParser` reads the `mapping.json` file and generates a `frames.json` item, which represents the synchronized 3D video sequence in the Dataloop studio.

In [ ]:
filters = dl.Filters(field=dl.FiltersKnownFields.NAME, values="mapping.json")
mapping_item = dataset.items.get_all_items(filters=filters)[0]
frames_item = LidarFileMappingParser().parse_data(mapping_item=mapping_item)
print(f"LiDAR Video File: ItemID: {frames_item.id}, ItemName: {frames_item.name}")

## <a id='upload-annotations'></a>6. Upload Annotations

With the `frames.json` video item created, you can now add annotations. The process is similar to annotating standard videos in Dataloop, but with support for 3D annotation types like cuboids. First, we need to retrieve the `frames.json` item we just created.

In [ ]:
filters = dl.Filters(field=dl.FiltersKnownFields.NAME, values="frames.json")
frames_item = dataset.items.get_all_items(filters=filters)[0]

### <a id='upload-cuboid'></a>6.1. Define and Upload a 3D Cuboid

#### Define the Annotation

To create a 3D cuboid annotation, we define its geometric properties and any additional metadata. The key parameters are:

*   **Position:** The 3D coordinates of the cuboid's center (x, y, z).
*   **Rotation:** Euler angles for the cuboid's orientation (roll, pitch, yaw).
*   **Scale:** The dimensions of the cuboid (width, height, depth).

The code below defines a `dl.Cube3d` annotation for a "person".

In [ ]:
# Define Your 3D Masterpiece
label = "person"
position = [
    73.9,
    -5.9,
    0.79
]
rotation = [
    0.0,
    0.0,
    0.0
]
scale = [
    1.2,
    0.8,
    1.8
]

# Add Some Extra Details
attributes = {"age": "adult"}
description = "An adult person standing on the station"

# Put It All Together
annotation_definition = dl.Cube3d(
    label=label,
    position=position,
    scale=scale,
    rotation=rotation,
    attributes=attributes,
    description=description
)

#### Upload the Annotation

Now that the `dl.Cube3d` object is defined, we use an annotation `builder` to add it to the `frames.json` item. We specify the frame range (`frame_num` to `end_frame_num`) where the annotation should appear and then upload it to the platform.

In [ ]:
# Grab Your Digital Sculpting Tools
builder = frames_item.annotations.builder()

# Place It in Your Scene
frame_num = 0
end_frame_num = 1
object_id = "0"
metadata = {"user": {"isDistracted": True}}

builder.add(
    annotation_definition=annotation_definition,
    frame_num=frame_num,
    end_frame_num=end_frame_num,
    object_id=object_id,
    metadata=metadata
)

# Save Your Work
builder.upload()

### <a id='view-in-studio'></a>6.2. View in Dataloop Studio

Once the annotation is uploaded, you can open the LiDAR video item in the Dataloop platform to view it in the LiDAR Studio. The following command will open the item directly in your web browser.

In [ ]:
frames_item.open_in_web()

## <a id='conclusion'></a>7. Conclusion and Next Steps

Congratulations! You have successfully walked through the entire process of preparing a LiDAR dataset for the Dataloop platform. This included:

1. Setting up your Dataloop environment and project.
2. Structuring and uploading raw LiDAR data (PCDs, images, and mapping file).
3. Constructing a composite LiDAR video item using the Dataloop LiDAR SDK.
4. Creating and uploading a 3D cuboid annotation to the video item.

### Next Steps

From here, you can explore more advanced functionalities:

*   Explore the full capabilities of our [Dataloop LiDAR SDK](https://github.com/dataloop-ai-apps/dtlpy-lidar).
*   (Advanced) Try constructing a LiDAR video item using a [Custom LiDAR Parser](https://developers.dataloop.ai/tutorials/data_management/items_and_annotations/other_data_types/lidar/chapter#using-custom-lidar-parser) for data that doesn't fit the standard structure.